In [1]:


#0) instalar todas as dependencias
%pip install langchain langchain-community openai pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\Pichau\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [15]:
#1) Importaçao das bibliotecas necessárias
from langchain.agents import initialize_agent, AgentType
from langchain.chat_models import ChatOpenAI
from langchain.tools import Tool
from math import sqrt 
import creds
import zipfile
import pandas as pd
from io import TextIOWrapper
import re 

# Variável global para armazenar o dataframe
df_global = None

#2) Criação da função das ferramentas
def calculadora(expr: str) -> str:
    """
    Útil para fazer cálculos como mínimo, máximo, média, soma, mediana em colunas do dataframe. A sintaxe esperada é 'operacao(NOME DA COLUNA)'. Ex: min(VALOR TOTAL). **Sempre use o nome EXATO da coluna. Se não tiver certeza, use 'ListarColunas' primeiro.**
    """
    global df_global
    if df_global is None:
        return "O dataframe ainda não foi carregado. Por favor, carregue um arquivo .zip primeiro."

    # Regex para capturar a operação e o nome da coluna
    match = re.match(r'(min|max|mean|sum|median)\((.*?)\)', expr.strip().lower())

    if not match:
        return f"Erro de sintaxe na calculadora. Formato esperado: 'operacao(NOME DA COLUNA)'. Ex: min(VALOR TOTAL), Ex2:max(VALOR NOTA FISCAL), Ex3:median(QUANTIDADE) . Certifique-se de que a coluna exista."

    operation = match.group(1)
    column_name_raw = match.group(2).strip()

    # Tenta encontrar a coluna exata, tratando variações de maiúsculas/minúsculas ou espaços
    # Primeiro, tenta o nome exato
    if column_name_raw in df_global.columns:
        column_name = column_name_raw
    else: # Tenta encontrar uma correspondência ignorando case e espaços
        found_column = None
        for col in df_global.columns:
            if col.replace(" ", "").lower() == column_name_raw.replace(" ", "").lower():
                found_column = col
                break
        if found_column:
            column_name = found_column
        else:
            return f"Erro: Coluna '{column_name_raw}' não encontrada no dataframe. As colunas disponíveis são: {list(df_global.columns)}. Use 'ListarColunas' para ver os nomes exatos."

    if column_name not in df_global.columns:
        return f"Erro: A coluna '{column_name_raw}' não foi encontrada no dataframe. As colunas disponíveis são: {list(df_global.columns)}. Use 'ListarColunas' para verificar os nomes exatos."

    try:
        if operation == 'min':
            resultado = df_global[column_name].min()
        elif operation == 'max':
            resultado = df_global[column_name].max()
        elif operation == 'mean':
            resultado = df_global[column_name].mean()
        elif operation == 'sum':
            resultado = df_global[column_name].sum()
        elif operation == 'median':
            resultado = df_global[column_name].median()
        else: # Isso não deveria acontecer com o regex, mas para segurança
            return f"Operação '{operation}' não suportada pela calculadora."

        return f"Utilizando a calculadora, o resultado para '{operation}({column_name})' é: {resultado}"
    except TypeError: # Para colunas não numéricas
        return f"Erro: A coluna '{column_name}' não contém dados numéricos para a operação '{operation}'. Por favor, escolha uma coluna numérica."
    except Exception as e:
        return f"Erro inesperado ao calcular: {str(e)}. Verifique os dados ou a coluna."

def carregar_arquivos(arquivo_zip_path: str):
    """
    Quando necessário extrai os arquivos do formato .zip (Extrair,Extract,Descompactar,Unzip,Abrir,Open,Deszipar,Decompress,Desempacotar,Unarchive,Desarquivar)
    Exemplo: 'C:\\Users\Pichau\Downloads\\teste\\202401_NFs.zip'
    Exemplo2: 'C:\\Users\\ABC\\Documents\\arquivo.zip'
    Exemplo3: 'C:\\_______NOME.zip'
    """
    global df_global # Declara que você vai modificar a variável global
    grupo_csv = {}
    with zipfile.ZipFile(arquivo_zip_path, 'r') as zip_ref:
        for arquivos in zip_ref.namelist():
            if arquivos.endswith('.csv'):
                with zip_ref.open(arquivos) as arq:
                    # Adicionado errors='ignore' para tentar lidar com problemas de codificação
                    df = pd.read_csv(TextIOWrapper(arq,'utf-8', errors='ignore'))
                    grupo_csv[arquivos] = df
            else:
                print(f"Sem suporte para o arquivo: '{arquivos}' não é um arquivo .csv")


        sorted_keys = sorted(grupo_csv.keys())
        primeiro_df = grupo_csv[sorted_keys[0]]
        segundo_df = grupo_csv[sorted_keys[1]]
       
        df_global = pd.merge(primeiro_df, segundo_df, on='CHAVE DE ACESSO', how='left')
        return f"Arquivos carregados e merged com sucesso! DataFrame pronto para análise com {len(df_global)} linhas e {len(df_global.columns)} colunas. As colunas disponíveis são: {list(df_global.columns)}. **Agora, se precisar, use 'ListarColunas' ou diretamente a 'Calculadora' com a sintaxe correta.**"

def listar_colunas() -> str:
    """
    Lista todos os nomes de colunas EXATOS disponíveis no dataframe carregado. Use esta ferramenta SEMPRE que precisar saber os nomes corretos das colunas para usar na 'Calculadora' ou para responder perguntas sobre quais colunas existem. **Não precisa de nenhum input.**
    """
    global df_global
    if df_global is None:
        return "Nenhum dataframe foi carregado ainda. Por favor, carregue um arquivo .zip primeiro."
    return f"As colunas disponíveis no dataframe são: {list(df_global.columns)}"

#3) Criando a ferramenta
calc_tool = Tool(
    name="Calculadora",
    func=calculadora,
    description=calculadora.__doc__.strip() # Pega a docstring da função como descrição
)


zip_tool = Tool(
    name="carregar_arquivos",
    func=carregar_arquivos,
    description=carregar_arquivos.__doc__.strip() # Pega a docstring da função como descrição,
)


colunas_tool = Tool(
    name="ListarColunas",
    func=listar_colunas,
    description=listar_colunas.__doc__.strip() # Pega a docstring da função como descrição
)


In [16]:

# 4) Escolhe o modelo selecionado e configuração de temperatura
llm = ChatOpenAI(
    model="mistralai/mistral-7b-instruct",  # Exemplo: Mistral
    base_url="https://openrouter.ai/api/v1",
    temperature=0.1,
    api_key=(creds.api_key)
)

# 5) Inicializando o agente com a ferramenta
agent = initialize_agent(
    tools=[calc_tool,zip_tool],
    llm=llm,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True)


In [21]:
# 6)Executa
user_input = "Dado o arquivo localizado em 'C:\\Users\Pichau\Downloads\\teste\\202401_NFs.zip' qual o maior valor de 'VALOR TOTAL'?"

resposta = agent.run(user_input)
print(f"Resposta: {resposta}")



> Entering new AgentExecutor chain...
 Thought: I need to extract the given zip file, load the data, and find the maximum value of the 'VALOR TOTAL' column.

Action:
```
{
  "action": "carregar_arquivos",
  "action_input": "C:\\Users\\Pichau\\Downloads\\teste\\202401_NFs.zip"
}
```


Observation: Arquivos carregados e merged com sucesso! DataFrame pronto para análise com 565 linhas e 47 colunas. As colunas disponíveis são: ['CHAVE DE ACESSO', 'MODELO_x', 'SÉRIE_x', 'NÚMERO_x', 'NATUREZA DA OPERAÇÃO_x', 'DATA EMISSÃO_x', 'EVENTO MAIS RECENTE', 'DATA/HORA EVENTO MAIS RECENTE', 'CPF/CNPJ Emitente_x', 'RAZÃO SOCIAL EMITENTE_x', 'INSCRIÇÃO ESTADUAL EMITENTE_x', 'UF EMITENTE_x', 'MUNICÍPIO EMITENTE_x', 'CNPJ DESTINATÁRIO_x', 'NOME DESTINATÁRIO_x', 'UF DESTINATÁRIO_x', 'INDICADOR IE DESTINATÁRIO_x', 'DESTINO DA OPERAÇÃO_x', 'CONSUMIDOR FINAL_x', 'PRESENÇA DO COMPRADOR_x', 'VALOR NOTA FISCAL', 'MODELO_y', 'SÉRIE_y', 'NÚMERO_y', 'NATUREZA DA OPERAÇÃO_y', 'DATA EMISSÃO_y', 'CPF/CNPJ Emitente_y